In [1]:
# pip install transformers

In [2]:
# pip install torch torchvision torchaudio -U

In [3]:
# pip install flash-attn --no-build-isolation

In [4]:
# ! pip install flash_attn -U

In [5]:
from torch import bfloat16
import transformers

In [6]:
model_id = "../Phi3MOE"

In [7]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype=bfloat16,
    device_map='auto'
)


Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

In [8]:
model.eval()

PhiMoEForCausalLM(
  (model): PhiMoEModel(
    (embed_tokens): Embedding(32064, 4096)
    (layers): ModuleList(
      (0-31): 32 x PhiMoEDecoderLayer(
        (self_attn): PhiMoESdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=True)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=True)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (rotary_emb): Phi3LongRoPEScaledRotaryEmbedding()
        )
        (block_sparse_moe): PhiMoESparseMoeBlock(
          (gate): Linear(in_features=4096, out_features=16, bias=False)
          (experts): ModuleList(
            (0-15): 16 x PhiMoEBlockSparseTop2MLP(
              (w1): Linear(in_features=4096, out_features=6400, bias=False)
              (w2): Linear(in_features=6400, out_features=4096, bias=False)
              (w3): Linear(in_features=4096, out_features=6400, bias=False)
   

In [9]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

In [10]:
# generate_text = transformers.pipeline(
#     model=model, tokenizer=tokenizer,
#     return_full_text=False,  # if using langchain set True
#     task="text-generation",
#     # we pass model parameters here too
#     temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
#     top_p=0.15,  # select from top tokens whose probability add up to 15%
#     top_k=0,  # select from top 0 tokens (because zero, relies on top_p)
#     max_new_tokens=2048,  # max number of tokens to generate in the output
#     repetition_penalty=1.1  # if output begins repeating increase
# )

pipe = transformers.pipeline(

    "text-generation",

    model=model,

    tokenizer=tokenizer,

)

2024-08-20 04:51:54.431510: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-20 04:51:54.431543: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
The model 'PhiMoEForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM'

In [11]:
generation_args = {

    "max_new_tokens": 512,

    "return_full_text": False,

    "temperature": 0.3,

    "do_sample": False,

}

In [12]:
sys_msg = """You are a helpful AI assistant, you are an agent capable of using a variety of tools to answer a question. Here are a few of the tools available to you:

- Blog: This tool helps you describe a certain knowledge point and content, and finally write it into Twitter or Facebook style content
- Translate: This is a tool that helps you translate into any language, using plain language as required

To use these tools you must always respond in JSON format containing `"tool_name"` and `"input"` key-value pairs. For example, to answer the question, "Build Muliti Agents with MOE models" you must use the calculator tool like so:

```json

{
    "tool_name": "Blog",
    "input": "Build Muliti Agents with MOE models"
}

```

Or to translate the question "can you introduce yourself in Chinese" you must respond:

```json

{
    "tool_name": "Search",
    "input": "can you introduce yourself in Chinese"
}

```

Remember just output the final result, ouput in JSON format containing `"agentid"`,`"tool_name"` , `"input"` and `"output"`  key-value pairs .:

```json

[


{   "agentid": "step1",
    "tool_name": "Blog",
    "input": "Build Muliti Agents with MOE models",
    "output": "........."
},

{   "agentid": "step2",
    "tool_name": "Search",
    "input": "can you introduce yourself in Chinese",
    "output": "........."
},
{
    "agentid": "final"
    "tool_name": "Result",
    "output": "........."
}
]

```

The users answer is as follows.
"""

In [13]:
def instruction_format(sys_message: str, query: str):
    # note, don't "</s>" to the end
    return f'<|system|> {sys_message} <|end|>\n<|user|> {query} <|end|>\n<|assistant|>'

In [14]:
query ='Write something about Generative AI with MOE , translate it to Chinese'

In [15]:
input_prompt = instruction_format(sys_msg, query)



In [16]:
input_prompt

'<|system|> You are a helpful AI assistant, you are an agent capable of using a variety of tools to answer a question. Here are a few of the tools available to you:\n\n- Blog: This tool helps you describe a certain knowledge point and content, and finally write it into Twitter or Facebook style content\n- Translate: This is a tool that helps you translate into any language, using plain language as required\n\nTo use these tools you must always respond in JSON format containing `"tool_name"` and `"input"` key-value pairs. For example, to answer the question, "Build Muliti Agents with MOE models" you must use the calculator tool like so:\n\n```json\n\n{\n    "tool_name": "Blog",\n    "input": "Build Muliti Agents with MOE models"\n}\n\n```\n\nOr to translate the question "can you introduce yourself in Chinese" you must respond:\n\n```json\n\n{\n    "tool_name": "Search",\n    "input": "can you introduce yourself in Chinese"\n}\n\n```\n\nRemember just output the final result, ouput in JSO

In [17]:
import torch

torch.cuda.empty_cache() 

import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True "

In [18]:
# res = generate_text(input_prompt)

output = pipe(input_prompt, **generation_args)

/anaconda/envs/azureml_py38/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


In [19]:
output[0]['generated_text']

' ```json\n\n[\n\n{   "agentid": "step1",\n    "tool_name": "Blog",\n    "input": "Generative AI with MOE",\n    "output": "Generative AI with MOE (Mixture of Experts) is a powerful approach that combines the strengths of generative models and the flexibility of MOE architecture. This hybrid model can generate high-quality, diverse, and contextually relevant content, making it suitable for various applications such as content creation, data augmentation, and more."\n},\n\n{   "agentid": "step2",\n    "tool_name": "Translate",\n    "input": "Generative AI with MOE is a powerful approach that combines the strengths of generative models and the flexibility of MOE architecture. This hybrid model can generate high-quality, diverse, and contextually relevant content, making it suitable for various applications such as content creation, data augmentation, and more.",\n    "output": "基于生成AI的MOE（Mixture of Experts）是一种强大的方法，它结合了生成模型的优势和MOE架构的灵活性。这种混合模型可以生成高质量、多样化且上下文相关的内容，使其适用于各种应用，如内容创建、数据增强等。"